### Шаг 1. Выгрузка данных

In [3]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Первым делом бросается в глаза отрицательные значеня в days_employed , у всех значение отрицательное ,кроме пенсионеров,у которых количество дней выходит за рамки разумного.Видно ,что пропуски допущеный в days_employed и total_income. Также сразу видна проблема с регистрами.Типы переменных везде аддекватные

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
income_types = ['безработный','в декрете','госслужащий','компаньон','пенсионер'\
                ,'предприниматель','сотрудник','студент']
for income_type in income_types:
    median_income = data[data['income_type'] == income_type]['total_income'].median()
    median_days_employed = data[data['income_type'] == income_type]['days_employed'].median()
    data['total_income'] = data['total_income'].fillna(value = median_income)
    data['days_employed']= data['days_employed'].fillna(value = median_days_employed)

### Вывод

Пропусков в категориальных переменных ни в одном из столбцов не обнаруженно.Обнаружены пропуски в столбцах: days_employed и total_income. Категоризируем зарплаты в каждой профессии и вычислияем медиану именно по каждой из них , заполняем по ним пропуски в обоих столбцах. Для проверки выводим информацию о значениях , видим ,что все пропуски успешно заполнены

### Замена типа данных

In [5]:
# float to int
data['total_income'] = data['total_income'].astype('int')

# Вывод

Переводим данные о ежемесячном доходе в целый тип ,чтобы сократить время работы программы и удобнее работать с методами sum() и остальными. Данные в таком количестве не сильно пострадают в статистике при опускании части после запятой

### Обработка дубликатов

In [6]:
#рассмотрим каждый стринг столбец методом value_counts()
data['education'] = data['education'].str.lower()
data['education'].value_counts()
#Теперь все education разные

#все разные,но Не женат / не замужем , начинается с верхнего регистра - справим
data.loc[data['family_status'] == 'Не женат / не замужем','family_status'] = 'не женат / не замужем'
data['family_status'].value_counts()

#children

data.loc[data['children'] < 0,'children'] = 0

#кол-во дупликатов 71 data.duplicated().sum()

data = data.drop_duplicates()

data.duplicated().sum()

0

### Вывод

Привели категориальные параметры к единому виду ,чтобы не было разницы с регистром тем самым при сравнении не было разницы между "среднее" и "СРЕДНЕЕ" итп,также для красоты таблицы , сделали все семейные статусы с маленькой буквы. Последующие дупликаты могут вылезти после лемматизации в столбце purpose

### Лемматизация

In [7]:
from pymystem3 import Mystem
m = Mystem()

def grouped_purpose(purpose):
    purposes = ['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        if lemma in purposes:
            return lemma

data['purpose'] = data['purpose'].apply(grouped_purpose)

### Вывод

Пишем функцию ,которая принимает на вход значение data['purpose'] и лемматезирует его ,чтобы выудить ключевое слово для понятия причины кредита.Присваиваем все ключевые слова в массив и сравниваем по нему на сходимость причины, в итоге возвразает это значение ,которое мы перезаписываем на прошлое ,чтобы не создавать новый столбец с еще 20000+ новыми данными

### Категоризация данных

In [8]:
#print(data.groupby('family_status')['children'].count())
#print(data.groupby('children')['children'].count())
#print(data.groupby('purpose')['children'].count())
print(data.groupby('children')['children'].count())
data.loc[data['children'] > 3 ,'children'] = 3
child_relation = data[['children','debt']]
fam_stat_relation = data[['family_status','debt']]
income_relation = data[['total_income','debt']]
purpose_relation = data[['purpose','debt']]

children
0     14138
1      4808
2      2052
3       330
4        41
5         9
20       76
Name: children, dtype: int64


### Вывод

Взял словари для быстрого отношения одного к другому ,чтобы можно было в ответе на вопрос с помощью метода вызвать отношение одного к другому. Также проверил , что основное количество данных приходится на 0 , 1 , 2 количество детей,что делает их самыми весомыми показателями.Поэтому показатели с меньшими количеством данных группируем под общее количество детей 3 

### Шаг 3. Ответьте на вопросы

In [9]:
#child_relation.groupby('children').mean().sort_values(by='debt',ascending = False)
pivot_child_relation = data.pivot_table(index ='children', values = 'debt')
pivot_child_relation.sort_values(by = 'debt', ascending = False)

,debt
children,
2,0.094542
1,0.092346
3,0.085526
0,0.075258


### Вывод

Чем больше детей ,тем сложнее закрыть кредит в срок , группа людей , у кого детей 3 и более(собраны под кол-вом детей = 3) показывают относительно хорошие результаты , но большое влияение на их среднее оказывает также у кого было 5 детей , ведь данных мало , а показатели все идеальные ,что делает среднее значение искаженным 

In [10]:
#fam_stat_relation.groupby('family_status').mean().sort_values(by='debt',ascending = False)
pivot_stat_relation = data.pivot_table(index = 'family_status', values = 'debt')
pivot_stat_relation.sort_values(by = 'debt', ascending = False)

,debt
family_status,
не женат / не замужем,0.097509
гражданский брак,0.093471
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


### Вывод

Семейный статус показывает ,что людям не женатым/не замужем сложнее всего закрыть кредит. В гражданском браке также данные показатель желает оставлять лучшего , возможно это связано с их будущей свадьбой и покупок ,что оставляет их в больших долгах. Женатым и замужем уже не нужны такие большие кредиты , и они уже могут распределять общие расходы 2ух человек. Людям в разводе многого не надо , по крайне мере ,пока они снова не станут состоять в гражданском браке и не влезут в большие кредиты. Вдовцам и вдовам подавно не нужны какие-то роскоши или крупые кредиты,что делает их самыми платежеспособными людьми 

In [11]:
#income_relation.groupby('total_income').mean().sort_values(by='debt',ascending = False)
pivot_income_relation = data.pivot_table(index = 'total_income', values = 'debt')
pivot_income_relation.sort_values(by = 'debt', ascending = False)

,debt
total_income,
20667,1.0
171879,1.0
263793,1.0
263633,1.0
82940,1.0
...,...
119484,0.0
119485,0.0
119488,0.0


### Вывод

Ежемесячный заработок самый прямой показатель на платежеспосбность ,ведь без денег - нет и средств на погашение кредита. Здесь статистика показывает ,что зависимость абсолютно прямая

In [12]:
#purpose_relation.groupby('purpose').mean().sort_values(by='debt',ascending = False)
pivot_purpose_relation = data.pivot_table(index = 'purpose', values = 'debt')
pivot_purpose_relation.sort_values(by = 'debt', ascending = False)

,debt
purpose,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
недвижимость,0.074634
жилье,0.069058


### Вывод

Зависимость от цели кредита влияет в обратном от логики понятии , то есть чем дороже/больше займ , тем чаще люди возвращают его в срок ,но это также может быть связанно с количеством времени ,которое дается на выплату и люди с течением времени уже привыкают к ежемесячной выплате в определенную сумму , когда машина или образование - быстрые кредиты , к которым нужно приспасабливаться и нету времени на ошибки в финансах ,что в обратном случае может привести к задержке в оплате в срок кредита

### Шаг 4. Общий вывод

Платежеспосбность однозначно зависит от количества детей , чем меньше их , тем лучше показатели выплаты в срок. Также заработная плата также очевидно влияет на платежеспособность клиента , ведь чем больше денег он получает ,тем легче ему будет выплатить данный кредит . Семейный статус и цель кредита не так однозначны ,но также видна логика , когда у людей по их семейному статусу предстоит измена ,у них и меняются цели кредита , тем самым делает их менее платежеспособными , ведь свадьба дело не дешевое,которое нужно успеть выплатить не в такой долгий срок , как жилье , так что для таких категорических параметров не хватает параметров на какое время они берут кредит , что на прямую зависит от их платежеспобности в срок.
<br>
Значит можем выделить гипотезы : 
<br>
1)количество детей напрямую влияет на на платежеспобность 
<br>
2)заработная плата также имеет прямое влияение на платежеспобность клиентта
<br>
3)люди ,которые женаты/замужем или были женаты/замужем имеют лучшую платежоспобность
<br>
4)чем дороже цель кредита , тем лучше у людей получается вернуть деньги в срок